# Tutorial 5: Data Indexing - B+ Trees and Extendible Hashing

## B+ Tree: Key Concepts

- **Internal Nodes**: Non-leaf nodes that guide searches using separator keys
- **Keys**: Values used for indexing and searching
- **Children**: Nodes that an internal node points to
- **Leaf Nodes**: Bottom-level nodes containing actual data or data pointers

## Problem 1: B+ Tree Construction and Insertion

**Task**: Construct a B+ tree by inserting (1, 3, 5, 7, 9, 2, 4, 6, 8, 10) sequentially, where each node can hold up to 4 pointers.

### Step-by-Step Construction:

1. Insert 1: Tree: [1]
2. Insert 3: Tree: [1, 3]
3. Insert 5: Tree: [1, 3, 5]
4. Insert 7: Split needed
 ```
Root: [3]
Leaves: [1, 3] ---> [5, 7]
 ```
5. Insert 9:
 ```
Root: [3]
Leaves: [1, 3] ---> [5, 7, 9]
 ```
6. Insert 2:
 ```
Root: [3]
Leaves: [1, 2, 3] ---> [5, 7, 9]
 ```
7. Insert 4, split needed:
 ```
Root: [3, 5]
Leaves: [1, 2, 3] ---> [4, 5] ---> [7, 9]
 ```
8. Insert 6:
 ```
Root: [3, 5]
Leaves: [1, 2, 3] ---> [4, 5] ---> [6, 7, 9]
 ```
9. Insert 8:
 ```
Root: [3, 5, 7]
Leaves: [1, 2, 3] ---> [4, 5] ---> [6, 7] ---> [8, 9]
 ```
10. Insert 10:
 ```
 Root: [3, 5, 7]
 Leaves: [1, 2, 3] ---> [4, 5] ---> [6, 7] ---> [8, 9, 10]
 ```

In [17]:
from graphviz import Digraph
import os
from PIL import Image
class BPlusTreeNode:
    def __init__(self, is_leaf=False):
        self.keys = []  # 存储键值
        self.children = []  # 存储子节点（仅内部节点使用）
        self.is_leaf = is_leaf  # 是否是叶子节点
        self.next_leaf = None  # 指向下一个叶子节点（仅叶子节点使用）

class BPlusTree:
    def __init__(self, max_keys=3):
        self.root = BPlusTreeNode(is_leaf=True)  # 根节点初始化为叶子节点
        self.max_keys = max_keys  # 每个节点最多存储的键值数

    def insert(self, key):
        """插入键值到 B+ 树"""
        root = self.root
        if len(root.keys) == self.max_keys:  # 如果根节点已满，分裂根节点
            new_root = BPlusTreeNode()
            self.root = new_root
            new_root.children.append(root)
            self._split_child(new_root, 0)
            self._insert_non_full(new_root, key)
        else:
            self._insert_non_full(root, key)

    def _insert_non_full(self, node, key):
        """在非满节点中插入键值"""
        if node.is_leaf:  # 如果是叶子节点，直接插入
            self._insert_into_leaf(node, key)
        else:  # 如果是内部节点，递归插入
            i = len(node.keys) - 1
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == self.max_keys:  # 如果子节点已满，分裂子节点
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], key)

    def _insert_into_leaf(self, node, key):
        """在叶子节点中插入键值"""
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1
        node.keys.insert(i, key)

    def _split_child(self, parent, index):
        """分裂子节点"""
        max_keys = self.max_keys
        child = parent.children[index]
        new_node = BPlusTreeNode(is_leaf=child.is_leaf)

        # 分裂键值和子节点
        mid = max_keys // 2
        parent.keys.insert(index, child.keys[mid])  # 将中间键插入父节点
        new_node.keys = child.keys[mid + 1:]  # 新节点存储右半部分键值
        child.keys = child.keys[:mid]  # 原节点存储左半部分键值

        if not child.is_leaf:  # 如果不是叶子节点，分裂子节点
            new_node.children = child.children[mid + 1:]
            child.children = child.children[:mid + 1]
        else:  # 如果是叶子节点，连接新的叶子节点
            new_node.next_leaf = child.next_leaf
            child.next_leaf = new_node

        parent.children.insert(index + 1, new_node)

    def visualize(self):
        """使用 graphviz 可视化 B+ 树"""
        dot = Digraph(comment='B+ Tree')
        self._add_nodes_to_graph(dot, self.root, None)
        return dot

    def _add_nodes_to_graph(self, dot, node, parent_key):
        """递归地将节点添加到 graphviz 图中"""
        # 为当前节点创建一个图形节点
        label = f"Keys: {node.keys}"
        if node.is_leaf:
            label += " (Leaf)"
        dot.node(str(id(node)), label=label)

        # 如果有父节点，添加边
        if parent_key is not None:
            dot.edge(str(parent_key), str(id(node)))

        # 递归处理子节点
        if not node.is_leaf:
            for child in node.children:
                self._add_nodes_to_graph(dot, child, str(id(node)))

# 测试代码
keys = [1, 3, 5, 7, 9, 2, 4, 6, 8, 10]
bplus_tree = BPlusTree(max_keys=3)

for key in keys:
    bplus_tree.insert(key)  # 插入键值

# 可视化 B+ 树
graph = bplus_tree.visualize()
png_name = graph.render("bplus_tree", format="png", cleanup=True)  # 保存为 PNG 文件

os.system(f"open { png_name } " if os.uname().sysname == "Darwin" else f"xdg-open { png_name }")


0

In [5]:
from graphviz import Digraph

dot = Digraph()
dot.node('A', 'Hello')
dot.node('B', 'World')
dot.edge('A', 'B')
dot.render('test-output', format='png', cleanup=True)  # 生成 test-output.png 文件
print("Graphviz test completed!")

Graphviz test completed!


## Problem 2: B+ Tree Deletion

Starting with our final tree from Problem 1, perform the following deletions:
### a) Delete 9
 ```
Root: [3, 5, 7]
Leaves: [1, 2, 3] ---> [4, 5] ---> [6, 7] ---> [8, 10]
 ```
### b) Delete 7
 ```
Root: [3, 5, 8]
Leaves: [1, 2, 3] ---> [4, 5] ---> [6] ---> [8, 10]
 ```
### c) Delete 8
Underflow in the last leaf node leads to redistribution or merging:
 ```
Root: [3, 5]
Leaves: [1, 2, 3] ---> [4, 5] ---> [6, 10]
 ```

## Problem 3: Extendible Hashing

**Task**: Use extendible hashing with bucket size 3 to hash: 16, 4, 6, 22, 24, 10, 31, 7, 9, 20, 26.
The hash function returns X LSBs (Least Significant Bits), where X is the global depth.

### Binary Representation:
- 16: 10000
- 4: 00100
- 6: 00110
- 22: 10110
- 24: 11000
- 10: 01010
- 31: 11111
- 7: 00111
- 9: 01001
- 20: 10100
- 26: 11010

### Insertion Process:

1. Insert 16, 4, 6:
 ```
Directory: 0 -> [16, 4, 6], 1 -> []
 ```
2. Insert 22 (overflow, split bucket):
 ```
Directory: 00 -> [16, 4], 01 -> [], 10 -> [6, 22], 11 -> []
 ```
3. Insert 24, 10:
 ```
Directory: 00 -> [16, 4, 24], 01 -> [], 10 -> [6, 22, 10], 11 -> []
 ```
4. Insert 31, 7, 9:
 ```
Directory: 00 -> [16, 4, 24], 01 -> [9], 10 -> [6, 22, 10], 11 -> [31, 7]
 ```
5. Insert 20 (overflow, split bucket):
 ```
Directory: 000 -> [16, 24], 001 -> [], 010 -> [9], 011 -> [],
100 -> [4, 20], 101 -> [], 110 -> [6, 22, 10], 111 -> [31, 7]
 ```
6. Insert 26:
 ```
Directory: 000 -> [16, 24], 001 -> [], 010 -> [9, 26], 011 -> [],
100 -> [4, 20], 101 -> [], 110 -> [6, 22, 10], 111 -> [31, 7]
 ```